# BDMM HW3 Modelling - Group 5

#### Made by:
* Catarina Candeias (m20200656@novaims.unl.pt)
* Catarina Urbano (m20200607@novaims.unl.pt)
* Margarida Pereira (m20201038@novaims.unl.pt) 
* Rita Ferreira (m20200661@novaims.unl.pt)
* Tiago Gonçalves (m20201053@novaims.unl.pt)

# Migrate all data to the database 

In [1]:
from pymongo import MongoClient
import sys, urllib.request, json, pymongo
from bson import json_util

In [2]:
host="rhea.isegi.unl.pt"
port="27022"
user="GROUP_05"
password="if8G1FFeOQICvBc4wz4ZYgft2wlxSnBR"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [3]:
db = client['WWI']

#### API with Application data

In [4]:
application_cities = "http://rhea.isegi.unl.pt:8080/Project_WWI_Application_Cities.json"
response = urllib.request.urlopen(application_cities)
data_cities = json_util.loads(response.read())

In [5]:
application_countries = "http://rhea.isegi.unl.pt:8080/Project_WWI_Application_Countries.json"
response = urllib.request.urlopen(application_countries)
data_countries = json_util.loads(response.read())

In [6]:
application_delivery = "http://rhea.isegi.unl.pt:8080/Project_WWI_Application_DeliveryMethods.json"
response = urllib.request.urlopen(application_delivery)
data_delivery = json_util.loads(response.read())

In [7]:
application_payment = "http://rhea.isegi.unl.pt:8080/Project_WWI_Application_PaymentMethods.json"
response = urllib.request.urlopen(application_payment)
data_payment = json_util.loads(response.read())

In [8]:
application_people = "http://rhea.isegi.unl.pt:8080/Project_WWI_Application_People.json"
response = urllib.request.urlopen(application_people)
data_people = json_util.loads(response.read())        

In [9]:
application_stateprovinces = "http://rhea.isegi.unl.pt:8080/Project_WWI_Application_StateProvinces.json"
response = urllib.request.urlopen(application_stateprovinces)
data_stateprovinces = json_util.loads(response.read())       

In [10]:
application_transactiontypes = "http://rhea.isegi.unl.pt:8080/Project_WWI_Application_TransactionTypes.json"
response = urllib.request.urlopen(application_transactiontypes)
data_transactiontypes = json_util.loads(response.read())       

In [11]:
#create collections
Application_Cities = db['Application_Cities']
Application_Countries = db['Application_Countries']
Application_DeliveryMethods = db['Application_DeliveryMethods']
Application_PaymentMethods = db['Application_PaymentMethods']
Application_People = db['Application_People']
Application_StateProvinces = db['Application_StateProvinces']
Application_TransactionTypes = db['Application_TransactionTypes']

# insert the data into each collection
Application_Cities.insert_many(data_cities)
Application_Countries.insert_many(data_countries)
Application_DeliveryMethods.insert_many(data_delivery)
Application_PaymentMethods.insert_many(data_payment)
Application_People.insert_many(data_people)
Application_StateProvinces.insert_many(data_stateprovinces)
Application_TransactionTypes.insert_many(data_transactiontypes)


#### RDBMS/SQL

In [12]:
host="rhea.isegi.unl.pt"
user="wwi-read-only-user"
password="jGp2GCqrss6nfTEu5ZawhW3mksLsQYQb"
database="WWI"

In [13]:
!pip install mysql-connector-python

In [34]:
import mysql.connector
mydb = mysql.connector.connect(host=host, user=user, database=database, password=password, port=3306)
mycursor = mydb.cursor()

mycursor.execute('SHOW TABLES;')
print(f"Tables:\n {mycursor.fetchall()}")

Tables:
 [('Purchasing_PurchaseOrderLines',), ('Purchasing_PurchaseOrders',), ('Purchasing_SupplierCategories',), ('Purchasing_SupplierTransactions',), ('Purchasing_Suppliers',), ('Sales_CustomerCategories',), ('Sales_CustomerTransactions',), ('Sales_Customers',), ('Sales_InvoiceLines',), ('Sales_Invoices',), ('Sales_OrderLines',), ('Sales_Orders',)]


In [37]:
import pandas as pd
import numpy as np
for collection in ['Purchasing_PurchaseOrderLines','Purchasing_PurchaseOrders','Purchasing_SupplierCategories','Purchasing_SupplierTransactions','Purchasing_Suppliers','Sales_CustomerCategories','Sales_CustomerTransactions','Sales_Customers','Sales_InvoiceLines','Sales_Invoices','Sales_OrderLines','Sales_Orders']:

    df = pd.read_sql(f"SELECT * FROM {collection} ", con=mydb)
    
    if collection == 'Sales_CustomerTransactions':
        df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
        df['FinalizationDate'] = pd.to_datetime(df['FinalizationDate'])
        
    if collection == 'Purchasing_PurchaseOrders':
        df['OrderDate'] = pd.to_datetime(df['OrderDate'])
        df['ExpectedDeliveryDate'] = pd.to_datetime(df['ExpectedDeliveryDate'])
    
    if collection == 'Purchasing_SupplierTransactions':
        df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
        df['FinalizationDate'] = pd.to_datetime(df['FinalizationDate'])
    
    if collection == 'Sales_Customers':
        df['AccountOpenedDate'] = pd.to_datetime(df['AccountOpenedDate'])
    
    if collection == 'Sales_Invoices':
        df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    
    if collection == 'Sales_Orders':
        df['OrderDate'] = pd.to_datetime(df['OrderDate'])
        df['ExpectedDeliveryDate'] = pd.to_datetime(df['ExpectedDeliveryDate'])
    
    df.replace({np.nan:None}, inplace=True)
    
    VARIAVEL = df.to_dict(orient='records')
    
    if collection == 'Purchasing_PurchaseOrderLines':
        Purchasing_PurchaseOrderLines = db[collection]
        Purchasing_PurchaseOrderLines.insert_many(VARIAVEL)

    if collection == 'Purchasing_PurchaseOrders':
        Purchasing_PurchaseOrders = db[collection]
        Purchasing_PurchaseOrders.insert_many(VARIAVEL)
        
    if collection == 'Purchasing_SupplierCategories':
        Purchasing_SupplierCategories = db[collection]
        Purchasing_SupplierCategories.insert_many(VARIAVEL)
        
    if collection == 'Purchasing_SupplierTransactions':
        Purchasing_SupplierTransactions = db[collection]
        Purchasing_SupplierTransactions.insert_many(VARIAVEL)
        
    if collection == 'Purchasing_Suppliers':
        Purchasing_Suppliers = db[collection]
        Purchasing_Suppliers.insert_many(VARIAVEL)
        
    if collection == 'Sales_CustomerCategories':
        Sales_CustomerCategories = db[collection]
        Sales_CustomerCategories.insert_many(VARIAVEL)
        
    if collection == 'Sales_CustomerTransactions':
        Sales_CustomerTransactions = db[collection]
        Sales_CustomerTransactions.insert_many(VARIAVEL)   
    
    if collection == 'Sales_Customers':
        Sales_Customers = db[collection]
        Sales_Customers.insert_many(VARIAVEL)
        
    if collection == 'Sales_InvoiceLines':
        Sales_InvoiceLines = db[collection]
        Sales_InvoiceLines.insert_many(VARIAVEL)
        
    if collection == 'Sales_Invoices':
        Sales_Invoices = db[collection]
        Sales_Invoices.insert_many(VARIAVEL)
        
    if collection == 'Sales_OrderLines':
        Sales_OrderLines = db[collection]
        Sales_OrderLines.insert_many(VARIAVEL)
        
    if collection == 'Sales_Orders':
        Sales_Orders = db[collection]
        Sales_Orders.insert_many(VARIAVEL)

#### CSV with Warehouse Data

In [16]:
# Import csv's

Warehouse_Colors = db['Warehouse_Colors']
Warehouse_PackageTypes = db['Warehouse_PackageTypes']
Warehouse_StockGroups = db['Warehouse_StockGroups']
Warehouse_StockItems = db['Warehouse_StockItems']
Warehouse_StockItemStockGroups = db['Warehouse_StockItemStockGroups']
Warehouse_StockItemTransactions = db['Warehouse_StockItemTransactions']

#### Warehouse_Colors

In [17]:
data_colors = pd.read_csv('csv/Project_WWI_Warehouse_Colors.csv').to_dict(orient='records')
Warehouse_Colors.insert_many(data_colors)

In [18]:
list(Warehouse_Colors.find(limit=5))

[{'_id': ObjectId('60b25fd91ab445ac8bcbdef0'),
  'ColorID': 1,
  'ColorName': 'Azure'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdef1'),
  'ColorID': 2,
  'ColorName': 'Beige'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdef2'),
  'ColorID': 3,
  'ColorName': 'Black'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdef3'),
  'ColorID': 4,
  'ColorName': 'Blue'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdef4'),
  'ColorID': 5,
  'ColorName': 'Charcoal'}]

#### Warehouse_PackageTypes

In [19]:
data_packagetypes = pd.read_csv('csv/Project_WWI_Warehouse_PackageTypes.csv').to_dict(orient='records')
Warehouse_PackageTypes.insert_many(data_packagetypes)

In [20]:
list(Warehouse_PackageTypes.find(limit=5))

[{'_id': ObjectId('60b25fd91ab445ac8bcbdf14'),
  'PackageTypeID': 1,
  'PackageTypeName': 'Bag'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdf15'),
  'PackageTypeID': 2,
  'PackageTypeName': 'Block'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdf16'),
  'PackageTypeID': 3,
  'PackageTypeName': 'Bottle'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdf17'),
  'PackageTypeID': 4,
  'PackageTypeName': 'Box'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdf18'),
  'PackageTypeID': 5,
  'PackageTypeName': 'Can'}]

#### Warehouse_StockGroups

In [21]:
data_stockgroups = pd.read_csv('csv/Project_WWI_Warehouse_StockGroups.csv').to_dict(orient='records')
Warehouse_StockGroups.insert_many(data_stockgroups)

In [22]:
list(Warehouse_StockGroups.find(limit=5))

[{'_id': ObjectId('60b25fd91ab445ac8bcbdf22'),
  'StockGroupID': 1,
  'StockGroupName': 'Novelty Items'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdf23'),
  'StockGroupID': 2,
  'StockGroupName': 'Clothing'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdf24'),
  'StockGroupID': 3,
  'StockGroupName': 'Mugs'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdf25'),
  'StockGroupID': 4,
  'StockGroupName': 'T-Shirts'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdf26'),
  'StockGroupID': 5,
  'StockGroupName': 'Airline Novelties'}]

#### Warehouse_StockItems

In [23]:
data_stockitems = pd.read_csv('csv/Project_WWI_Warehouse_StockItems.csv').replace({np.nan:None}).to_dict(orient='records')
Warehouse_StockItems.insert_many(data_stockitems)

In [24]:
list(Warehouse_StockItems.find(limit=5))

[{'_id': ObjectId('60b25fd91ab445ac8bcbdf2c'),
  'StockItemID': 1,
  'StockItemName': 'USB missile launcher (Green)',
  'SupplierID': 12,
  'ColorID': None,
  'UnitPackageID': 7,
  'OuterPackageID': 7,
  'Brand': None,
  'Size': None,
  'LeadTimeDays': 14,
  'QuantityPerOuter': 1,
  'IsChillerStock': False,
  'Barcode': None,
  'TaxRate': 15.0,
  'UnitPrice': 25.0,
  'RecommendedRetailPrice': 37.38,
  'TypicalWeightPerUnit': 0.3,
  'MarketingComments': 'Complete with 12 projectiles',
  'InternalComments': None,
  'Photo': None,
  'SearchDetails': 'USB missile launcher (Green) Complete with 12 projectiles'},
 {'_id': ObjectId('60b25fd91ab445ac8bcbdf2d'),
  'StockItemID': 2,
  'StockItemName': 'USB rocket launcher (Gray)',
  'SupplierID': 12,
  'ColorID': 12.0,
  'UnitPackageID': 7,
  'OuterPackageID': 7,
  'Brand': None,
  'Size': None,
  'LeadTimeDays': 14,
  'QuantityPerOuter': 1,
  'IsChillerStock': False,
  'Barcode': None,
  'TaxRate': 15.0,
  'UnitPrice': 25.0,
  'RecommendedRetai

#### Warehouse_StockItemStockGroups

In [25]:
data_stockitemstockgroups = pd.read_csv('csv/Project_WWI_Warehouse_StockItemStockGroups.csv').to_dict(orient='records')
Warehouse_StockItemStockGroups.insert_many(data_stockitemstockgroups)

In [26]:
list(Warehouse_StockItemStockGroups.find(limit=5))

[{'_id': ObjectId('60b25fd91ab445ac8bcbe00f'),
  'StockItemStockGroupID': 1,
  'StockItemID': 1,
  'StockGroupID': 6},
 {'_id': ObjectId('60b25fd91ab445ac8bcbe010'),
  'StockItemStockGroupID': 2,
  'StockItemID': 1,
  'StockGroupID': 1},
 {'_id': ObjectId('60b25fd91ab445ac8bcbe011'),
  'StockItemStockGroupID': 3,
  'StockItemID': 1,
  'StockGroupID': 7},
 {'_id': ObjectId('60b25fd91ab445ac8bcbe012'),
  'StockItemStockGroupID': 4,
  'StockItemID': 2,
  'StockGroupID': 6},
 {'_id': ObjectId('60b25fd91ab445ac8bcbe013'),
  'StockItemStockGroupID': 5,
  'StockItemID': 2,
  'StockGroupID': 1}]

#### Warehouse_StockItemTransactions

In [27]:
data_stockitemtransactions = pd.read_csv('csv/Project_WWI_Warehouse_StockItemTransactions.csv').replace({np.nan:None})
data_stockitemtransactions['TransactionOccurredWhen'] = pd.to_datetime(pd.read_csv('csv/Project_WWI_Warehouse_StockItemTransactions.csv')['TransactionOccurredWhen'])
data_stockitemtransactions = data_stockitemtransactions.to_dict(orient='records')
Warehouse_StockItemTransactions.insert_many(data_stockitemtransactions)

In [28]:
Warehouse_StockItemTransactions.count_documents({})

236667

In [29]:
list(Warehouse_StockItemTransactions.find(limit=5))

[{'_id': ObjectId('60b25fde1ab445ac8bcbe1c9'),
  'StockItemTransactionID': 1,
  'StockItemID': 67,
  'TransactionTypeID': 10,
  'CustomerID': 832.0,
  'InvoiceID': 1.0,
  'SupplierID': None,
  'PurchaseOrderID': None,
  'TransactionOccurredWhen': datetime.datetime(2013, 1, 1, 12, 0),
  'Quantity': -10.0},
 {'_id': ObjectId('60b25fde1ab445ac8bcbe1ca'),
  'StockItemTransactionID': 3,
  'StockItemID': 50,
  'TransactionTypeID': 10,
  'CustomerID': 803.0,
  'InvoiceID': 2.0,
  'SupplierID': None,
  'PurchaseOrderID': None,
  'TransactionOccurredWhen': datetime.datetime(2013, 1, 1, 12, 0),
  'Quantity': -9.0},
 {'_id': ObjectId('60b25fde1ab445ac8bcbe1cb'),
  'StockItemTransactionID': 4,
  'StockItemID': 10,
  'TransactionTypeID': 10,
  'CustomerID': 803.0,
  'InvoiceID': 2.0,
  'SupplierID': None,
  'PurchaseOrderID': None,
  'TransactionOccurredWhen': datetime.datetime(2013, 1, 1, 12, 0),
  'Quantity': -9.0},
 {'_id': ObjectId('60b25fde1ab445ac8bcbe1cc'),
  'StockItemTransactionID': 6,
  '

# Dealing with the null values - Removing all fields that are null from the respective collection

#### Remove Nulls from Application_People Collection:

```
db.Application_People.update(
  {},
  [{ $replaceWith: {
    $arrayToObject: {
      $filter: {
        input: { $objectToArray: "$$ROOT" },
        as: "item",
        cond: { $ne: ["$$item.v", null] }
      }
    }
  }}],
  { multi: true }
)

```

#### Remove Nulls from Warehouse_StockItems Collection:

```
db.Warehouse_StockItems.update(
  {},
  [{ $replaceWith: {
    $arrayToObject: {
      $filter: {
        input: { $objectToArray: "$$ROOT" },
        as: "item",
        cond: { $ne: ["$$item.v", null] }
      }
    }
  }}],
  { multi: true }
)

```

#### Remove Nulls from  Warehouse_StockItemTransactions Collection:

```
db.Warehouse_StockItemTransactions.update(
  {},
  [{ $replaceWith: {
    $arrayToObject: {
      $filter: {
        input: { $objectToArray: "$$ROOT" },
        as: "item",
        cond: { $ne: ["$$item.v", null] }
      }
    }
  }}],
  { multi: true }
)

```

#### Remove Nulls from  Purchasing_PurchaseOrders Collection:

```
db.Purchasing_PurchaseOrders.update(
  {},
  [{ $replaceWith: {
    $arrayToObject: {
      $filter: {
        input: { $objectToArray: "$$ROOT" },
        as: "item",
        cond: { $ne: ["$$item.v", null] }
      }
    }
  }}],
  { multi: true }
)
``` 

#### Remove Nulls from Purchasing_Suppliers Collection:

```
db.Purchasing_Suppliers.update(
  {},
  [{ $replaceWith: {
    $arrayToObject: {
      $filter: {
        input: { $objectToArray: "$$ROOT" },
        as: "item",
        cond: { $ne: ["$$item.v", null] }
      }
    }
  }}],
  { multi: true }
)

```

#### Remove Nulls from Sales_CustomerTransactions Collection:

```
db.Sales_CustomerTransactions.update(
  {},
  [{ $replaceWith: {
    $arrayToObject: {
      $filter: {
        input: { $objectToArray: "$$ROOT" },
        as: "item",
        cond: { $ne: ["$$item.v", null] }
      }
    }
  }}],
  { multi: true }
)

```

#### Remove Nulls from Sales_Customers Collection:

```
db.Sales_Customers.update(
  {},
  [{ $replaceWith: {
    $arrayToObject: {
      $filter: {
        input: { $objectToArray: "$$ROOT" },
        as: "item",
        cond: { $ne: ["$$item.v", null] }
      }
    }
  }}],
  { multi: true }
)


```

#### Remove Nulls from Sales_Invoices Collection:

```
db.Sales_Invoices.update(
  {},
  [{ $replaceWith: {
    $arrayToObject: {
      $filter: {
        input: { $objectToArray: "$$ROOT" },
        as: "item",
        cond: { $ne: ["$$item.v", null] }
      }
    }
  }}],
  { multi: true }
)

``` 

#### Remove Nulls from Sales_Orders Collection:

```
db.Sales_Orders.update(
  {},
  [{ $replaceWith: {
    $arrayToObject: {
      $filter: {
        input: { $objectToArray: "$$ROOT" },
        as: "item",
        cond: { $ne: ["$$item.v", null] }
      }
    }
  }}],
  { multi: true }
)

```

# Change Datatypes

PaymentMethodID to Double :

``` db.Purchasing_SupplierTransactions.find().forEach(function(data) { db.Purchasing_SupplierTransactions.update({_id:data._id},{$set:{PaymentMethodID:parseInt(data.PaymentMethodID)}}); }); ```

Quantity to Int :

``` db.Sales_OrderLines.find().forEach(function(data) { db.Sales_OrderLines.update({_id:data._id},{$set:{Quantity:NumberInt(data.Quantity)}}); }); ```


PurchaseOrderLineID to Int:

``` db.Purchasing_PurchaseOrderLines.find().forEach(function(data) { db.Purchasing_PurchaseOrderLines.update({_id:data._id},{$set:{PurchaseOrderLineID:NumberInt(data.PurchaseOrderLineID)}}); }); ```

ExpectedUnitPricePerOuter to Float:

``` db.Purchasing_PurchaseOrderLines.find().forEach(function(data) { db.Purchasing_PurchaseOrderLines.update({_id:data._id},{$set:{ExpectedUnitPricePerOuter:NumberDecimal (data.ExpectedUnitPricePerOuter)}}); }); ```

# New Variables

#### Sales_InvoiceLines - Adding Revenue (Quantity x Price)

In [78]:
db.Sales_InvoiceLines.aggregate( [
   {
    '$addFields': {
       'Revenue': { '$multiply': [ "$Quantity", "$UnitPrice" ] }
     }
   },
    
    {'$out':'Sales_InvoiceLines'}
    
])

In [79]:
db.Sales_InvoiceLines.count_documents({})

30000

#### Sales_Customers - Adding coords and GeoIndexing

In [62]:
from pymongo import GEO2D

In [60]:
db.Sales_Customers.aggregate( [
   {
    "$addFields": {
        "coords": ["$DeliveryLocationLat", "$DeliveryLocationLong"]
    }
   },
    
    {'$out':'Sales_Customers'}
    
])

In [63]:
db.Sales_Customers.create_index([("coords", GEO2D)])

'coords_2d'

# Embeddings

#### Sales_Invoice_Lines, Sales_Invoice and Application_People  - Query from Question 5

# IMAGEM 

In [4]:
query_1 = {
        "$lookup":{
           "from": "Application_People",
           "localField": "SalespersonPersonID",
           "foreignField": "PersonID",
           "as": "ApplicationPerson"
        }
    }

# Both queries 2 are equivalent, although the $merge operator has more functionalities
query_2 = { 
        "$merge" : {
            "into" : "Collection_Q5_Embedded_Invoice_People",
            "whenMatched": "merge"
        } 
    }

pipeline = [query_1,query_2]

In [6]:
db.Sales_Invoices.aggregate(pipeline)

In [7]:
query_1 = {
        "$lookup":{
           "from": "Collection_Q5_Embedded_Invoice_People",
           "localField": "InvoiceID",
           "foreignField": "InvoiceID",
           "as": "Invoice"
        }
    }


# Both queries 2 are equivalent, although the $merge operator has more functionalities
query_2 = { 
        "$merge" : {
            "into" : "Collection_Q5_Embedded_InvoiceLines_Invoice_People",
            "whenMatched": "merge"
        } 
    }

pipeline = [query_1,query_2]

In [8]:
db.Sales_InvoiceLines.aggregate(pipeline)

In [64]:
db['Collection_Q5_Embedded_Invoice_People'].drop()
# dropping intermediate collection

####  Sales_Order, Sales_OrderLines, Warehouse_StockItems - Query from Questions 2 and 3

# IMAGEM

In [10]:
query_1 = {
        "$lookup":{
           "from": "Sales_Orders",
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "Order"
        }
    }

query_2 = {
        "$lookup":{
           "from": "Warehouse_StockItems",
           "localField": "StockItemID",
           "foreignField": "StockItemID",
           "as": "StockItem"
        }
    }


# Both queries 2 are equivalent, although the $merge operator has more functionalities
query_3 = { 
        "$merge" : {
            "into" : "Collection_Q2_Q3_Embedded_OrderLines_Order_StockItems",
            "whenMatched": "merge"
        } 
    }

pipeline = [query_1,query_2, query_3]

In [11]:
db.Sales_OrderLines.aggregate(pipeline)

####  Applications_Countries, Applications_StateProvince, Applications_Cities - Q1

# IMAGEM

In [20]:
query_1 = {
        "$lookup":{
           "from": "Application_Countries",
           "localField": "CountryID",
           "foreignField": "CountryID",
           "as": "ApplicationCountry"
        }
    }

# Both queries 2 are equivalent, although the $merge operator has more functionalities
query_2 = { 
        "$merge" : {
            "into" : "Collection_Q1_StateProvince_Country",
            "whenMatched": "merge"
        } 
    }

pipeline = [query_1,query_2]

In [21]:
db.Application_StateProvinces.aggregate(pipeline)

In [22]:
query_1 = {
        "$lookup":{
           "from": "Collection_Q1_StateProvince_Country",
           "localField": "StateProvinceID",
           "foreignField": "StateProvinceID",
           "as": "StateProvince"
        }
    }


# Both queries 2 are equivalent, although the $merge operator has more functionalities
query_2 = { 
        "$merge" : {
            "into" : "Collection_Q1_Embedded_City_StateProvince_Country",
            "whenMatched": "merge"
        } 
    }

pipeline = [query_1,query_2]

In [23]:
db.Application_Cities.aggregate(pipeline)

In [65]:
db['Collection_Q1_StateProvince_Country'].drop()
# dropping intermediate collection

####  Purchasing_SupplierCategories, Purchasing_Suppliers, Purchasing_SupplierTransactions - Q4

# IMAGEM

In [16]:
query_1 = {
        "$lookup":{
           "from": "Purchasing_SupplierCategories",
           "localField": "SupplierCategoryID",
           "foreignField": "SupplierCategoryID",
           "as": "SupplierCategory"
        }
    }

# Both queries 2 are equivalent, although the $merge operator has more functionalities
query_2 = { 
        "$merge" : {
            "into" : "Collection_Q4_Supplier_SupplierCategory",
            "whenMatched": "merge"
        } 
    }

pipeline = [query_1,query_2]

In [17]:
db.Purchasing_Suppliers.aggregate(pipeline)

In [24]:
query_1 = {
        "$lookup":{
           "from": "Collection_Q4_Supplier_SupplierCategory",
           "localField": "SupplierID",
           "foreignField": "SupplierID",
           "as": "Supplier"
        }
    }


# Both queries 2 are equivalent, although the $merge operator has more functionalities
query_2 = { 
        "$merge" : {
            "into" : "Collection_Q4_Embedded_SupplierTransaction_Supplier_SupplierCategory",
            "whenMatched": "merge"
        } 
    }

pipeline = [query_1,query_2]

In [25]:
db.Purchasing_SupplierTransactions.aggregate(pipeline)

In [66]:
db['Collection_Q4_Supplier_SupplierCategory'].drop()
# dropping intermediate collection